In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0.5,
    base_url="http://localhost:11434",
    stream_usage=True  # 流模式需开启统计
)

https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [26]:
import uuid
from langchain.llms import OpenAI
from langchain_core.language_models.llms import BaseLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

class RunnableHistoryMemory:
    def __init__(self, history_function, model: BaseLLM, prompt: ChatPromptTemplate):
        """
        Initializes the encapsulation class
        :param history_function: The function used to retrieve history records
        :param model: The language model to be used (e.g., text-davinci-003)
        :param prompt: The chat prompt template
        """
        try:
            # Create chat prompt template
            self.prompt = prompt

            # Create runnable
            self.runnable = self.prompt | model

            # Encapsulate into RunnableWithMessageHistory with history
            self.runnable_with_history = RunnableWithMessageHistory(
                self.runnable,
                history_function,
                input_messages_key="input",
                history_messages_key="history",
            )
            
            # Create session
            self.session = self.generate_random_session()

        except TypeError as e:
            # Handle type errors (e.g., incorrect input types for prompt, model, or history_function)
            print(f"TypeError: {str(e)} - Please ensure all parameters are of the correct type (e.g., prompt, model, history_function)")
        
        except AttributeError as e:
            # Handle attribute errors (e.g., missing required attributes or methods)
            print(f"AttributeError: {str(e)} - Ensure all required attributes or methods exist and are initialized properly")
        
        except Exception as e:
            # Handle other unknown errors
            print(f"Unknown error: {str(e)} - There was an issue with the initialization, please check input parameters and configurations.")
            
    def generate_random_session(self):
        """
        Generates a random session ID
        :return: A randomly generated session ID (UUID)
        """
        try:
            # Generate a random session ID using UUID
            session_id = str(uuid.uuid4())
            return session_id
        
        except Exception as e:
            # Handle any potential errors in session ID generation
            print(f"Error generating session ID: {str(e)} - Please check the UUID generation process.")
            return None

    
    # def process_input(self, input_text: str, session_id: str,language: str = "english"):
    #     """
    #     处理输入文本并返回响应
    #     :param input_text: 用户输入的文本
    #     :param session_id: 会话 ID，用于获取历史记录
    #     :param language:语言类型，默认英语
    #     :return: 生成的响应
    #     """
    #     # 将输入和历史传递给 runnable
    #     response = self.runnable_with_history.invoke(
    #         {"language": language, "input": input_text},
    #         config={"configurable": {"session_id": session_id}})
    #     
    #     # 返回生成的响应
    #     return response
    def process_input(self, input_text: str, session_id: str = None, language: str = "english"):
        """
        Processes the input text and returns a response
        :param input_text: The text input from the user
        :param session_id: The session ID used for retrieving historical records
        :param language: The language type, default is English
        :return: The generated response
        """
        try:
            # Pass input and history to runnable
            if session_id != None:
                response = self.runnable_with_history.invoke(
                    {"language": language, "input": input_text},
                    config={"configurable": {"session_id": session_id}})
            else:
                response = self.runnable_with_history.invoke(
                    {"language": language, "input": input_text},
                    config={"configurable": {"session_id": self.session}})
        
            # Return the generated response
            return response
    
        except AttributeError as e:
            # Handle attribute errors (e.g., incorrect initialization of runnable_with_history)
            return {"error": f"AttributeError: {str(e)} - Please check if runnable_with_history is correctly initialized"}
    
        except TypeError as e:
            # Handle type errors (e.g., incorrect input type)
            return {"error": f"TypeError: {str(e)} - The input type might not be correct"}
    
        except Exception as e:
            # Handle other unknown errors
            return {"error": f"Unknown error: {str(e)} - Please check the input and system configuration"}
    
    

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# 创建封装类的实例
runnable_wrapper = RunnableHistoryMemory(get_session_history,llm,prompt)


session_id = "session_1"  # 示例会话 ID
input_text = "hi - im bob!"
response = runnable_wrapper.process_input(input_text, session_id)

# 输出响应
print(response)


content='<think>\nAlright, let me figure this out. The user just said "hi - im bob!" and I responded with "hi, bob!" in a shorter form. I\'m trying to see if there\'s anything more specific they want or if it\'s just testing how I handle short communications.\n\nMaybe they\'re curious about my response style. But since the last message was short, perhaps that\'s all I can offer without more context. It seems like a casual greeting, so maintaining simplicity is probably best.\n</think>\n\nHi, bob!' additional_kwargs={} response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-03-04T08:27:10.083294Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5113792500, 'load_duration': 567305167, 'prompt_eval_count': 559, 'prompt_eval_duration': 1167000000, 'eval_count': 108, 'eval_duration': 3133000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-c483cf03-3eb6-4558-86c7-f4eea0483803-0' usage_metadata={'input_tokens': 559, 'output_token

In [20]:
input_text = "whats my name?"
response = runnable_wrapper.process_input(input_text, session_id)

# 输出响应
print(response)

content='<think>\nAlright, let me figure this out. The user just asked "whats my name?" and I responded with "My name is Bob." I\'m trying to see if there\'s anything more specific they want or if it\'s just a straightforward reply.\n\nHmm, maybe they\'re testing how I handle short questions or if there\'s something particular about the response style. But without any additional context, it seems like a simple greeting would be appropriate.\n\nI should respond in a friendly and concise manner to maintain their comfort level with my responses.\n</think>\n\nMy name is Bob!' additional_kwargs={} response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-03-04T08:11:06.83964Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3429959500, 'load_duration': 29238125, 'prompt_eval_count': 325, 'prompt_eval_duration': 116000000, 'eval_count': 116, 'eval_duration': 3274000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-64ab67bc-e0ca-43c

In [29]:
input_text = "hi - im white!"
response2 = runnable_wrapper.process_input(input_text)

In [30]:
# 输出响应
print(response2)

content='<think>\nAlright, let\'s see what the user is asking here. They wrote "hi - im white!" and I responded with "Hi - I\'m white!". Hmm, that seems a bit too straightforward. Maybe they\'re testing if I understand their language choices.\n\nI should consider why they said "hi" instead of "hello" or "hey". It could be because they\'re using a different greeting style. They mentioned "I\'m white!", which is a common way to greet someone in a formal setting, like when talking about being white-sounding or in a casual context.\n\nThey might want me to respond in a similar tone but also check if I understand their language. Alternatively, maybe they just wanted a quick response without any specific instructions. I should make sure my reply is friendly and clear, using the same greeting style as before.\n\nSo, responding with "Hi - I\'m white!" keeps the conversation going smoothly and maintains the greeting. It\'s concise enough to stay within the 20-word limit they set but still conve

In [31]:
input_text = "whats my name?"
response = runnable_wrapper.process_input(input_text)

# 输出响应
print(response)

content='<think>\nOkay, so the user just asked, "whats my name?" after I responded with "Hi - I\'m white!" They probably want me to confirm their name in a friendly way. I should keep it simple and clear.\n\nMaybe they\'re checking if I understand them or if they need any additional help. It\'s good to be concise but polite, so I\'ll stick to the same greeting style as before.\n</think>\n\nHi - I\'m white!' additional_kwargs={} response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-03-04T08:28:47.953189Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2695623875, 'load_duration': 30086666, 'prompt_eval_count': 254, 'prompt_eval_duration': 119000000, 'eval_count': 94, 'eval_duration': 2539000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-0e15eb7a-fb54-4bc9-b353-ed02a8173c8c-0' usage_metadata={'input_tokens': 254, 'output_tokens': 94, 'total_tokens': 348}


In [32]:
input_text = "whats my name?"
response = runnable_wrapper.process_input(input_text,session_id)

# 输出响应
print(response)

content='<think>\nOkay, let me figure this out. The user just asked "whats my name?" and I responded with "My name is Bob." I\'m trying to see if there\'s anything more specific they want or if it\'s just a straightforward reply.\n\nMaybe they\'re curious about how I handle short questions or if there\'s something particular about the response style. But without any additional context, it seems like a simple greeting would be appropriate.\n</think>\n\nMy name is Bob!' additional_kwargs={} response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-03-04T08:29:11.87896Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4202590333, 'load_duration': 30250333, 'prompt_eval_count': 674, 'prompt_eval_duration': 1319000000, 'eval_count': 97, 'eval_duration': 2825000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-f8eba877-c4b7-4a7b-b7a5-688eb6bbfa5e-0' usage_metadata={'input_tokens': 674, 'output_tokens': 97, 'total_tokens': 771}
